In [ ]:
from collections import deque
import heapq


In [ ]:

class Board:
    def __init__(self,sequence):
        self.board = [sequence[:3], sequence[3:6], sequence[6:]]
        # convert a regular sequence to a 2d board
        self.prev = None
        # to keep track of the previous states

    def __eq__ (self, other):
        return self.board == other.board
        # compare two boards
    def __hash__(self):
        return hash(tuple(tuple(row) for row in self.board))
        #hash the board to be able to store it in a set
    def draw(self):
        for row in self.board:
            print(" | ".join(str(cell) for cell in row))
        print("-" * 9)
        # draw the board shape
    def __lt__(self, other):
        return self.board < other.board
        # compare two boards

    def copy(self):
        return Board([cell for row in self.board for cell in row])
        # copy the board

In [ ]:
goal = Board([0, 1, 2, 3, 4, 5, 6, 7, 8])
# the goal board

In [ ]:
def isthegoal(Board):
    return Board.__eq__(goal)
    # check if the board is the goal board

def possible_moves(Board):
    possible = []
    #find the 0
    row = 0
    col = 0
    for i in range(3):
        for j in range(3):
            if Board.board[i][j] == 0:
                row = i
                col = j
    for [i,j] in [[row-1, col], [row+1, col], [row, col-1], [row, col+1]]:
        if i >= 0 and j >= 0 and i < 3 and j < 3:
            # make a copy to don't edit the original board
            n = Board.copy()
            # swap the 0 with the neighbor
            n.board[row][col], n.board[i][j] = n.board[i][j], n.board[row][col]
            # add the new state to the possible moves
            possible.append(n)
    return possible

In [ ]:
# Example usage of possible_moves function
initial_board = Board([1,2,3,4,5,6,7,8,0])
print("Initial Board:")
initial_board.draw()
moves = possible_moves(initial_board)
print("Possible Moves:")
for move in moves:
    move.draw()


Initial Board:
1 | 2 | 3
4 | 5 | 6
7 | 8 | 0
---------
Possible Moves:
1 | 2 | 3
4 | 5 | 0
7 | 8 | 6
---------
1 | 2 | 3
4 | 5 | 6
7 | 0 | 8
---------


In [ ]:
def bfs(initial_board):
    done = False
    frontier = deque([initial_board])
    # queue of the states to be visited
    # The deque stands for double-ended queue provides an efficient way to add and remove elements from both ends of a sequence

    on_frontier = set([initial_board])
    # set of the states that are currently on the frontier
    visited = set()
    # set of the states that have been visited
    v = None
    # the current state
    while not done and len(frontier) > 0:
        v = frontier.popleft()
        # pops the oldest / shallowets state from the frontier
        visited.add(v)
        # add the state to the visited set
        on_frontier.remove(v)
        # remove the state from the frontier set
        if isthegoal(v):
            done = True
        else:
            for n in possible_moves(v):
                if n not in visited and n not in on_frontier:
                    # make sure n is not a repeated state and has not been visited
                    on_frontier.add(n)
                    n.prev = v
                    # set the previous state of n to be v
                    frontier.append(n)
                    # add n to the frontier
    solution = [v]
    # the solution list
    while v.prev:
        solution.append(v.prev)
        v = v.prev
    solution.reverse()
    return solution



# initial_board = Board([3,1,2,6,4,5,0,7,8])
print("Initial Board:")
initial_board.draw()
print("BFS")
solution = bfs(initial_board)

# for s in solution:
#     s.draw()

print("Depth")
print(len(solution))

Initial Board:
1 | 2 | 3
4 | 5 | 6
7 | 8 | 0
---------
BFS
Depth
23


In [ ]:
def dfs(initial_board):
    stack = [initial_board]
    # stack of the states to be visited
    visited = set()
    on_stack = set([initial_board])
    # set of the states that are currently on the stack
    while stack:
        v = stack.pop()
        # pop the newest state from the stack to follow its children
        on_stack.remove(v)
        visited.add(v)

        if isthegoal(v):
            solution = [v]
            while v.prev:
                solution.append(v.prev)
                v = v.prev
            solution.reverse()
            return solution

        for n in possible_moves(v):
            if n not in visited and n not in on_stack:
                n.prev = v
                stack.append(n)
                on_stack.add(n)
    return None


# initial_board = Board([1, 2, 3, 4, 0, 5, 6, 7, 8])
print("Initial Board:")
initial_board.draw()
print("DFS")
solution = dfs(initial_board)
# for s in solution:
#     s.draw()

print("Depth")
print(len(solution))

Initial Board:
1 | 2 | 3
4 | 5 | 6
7 | 8 | 0
---------
DFS
Depth
63141


In [ ]:
def iddfs(initial_state):
    def dls(node, depth):
        if depth == 0 and isthegoal(node):
            return [node]
        if depth > 0:
            for neighbor in possible_moves(node):
                neighbor.prev = node
                result = dls(neighbor, depth - 1)
                if result:
                    return [node] + [result] if isinstance(result, Board) else [node] + result
        return None

    depth = 0
    while True:
        result = dls(initial_state, depth)
        if result:
            return result
        depth += 1



# initial_board = Board([1, 2, 3, 4, 0, 5, 6, 7, 8])
print("Initial Board:")
initial_board.draw()
print("IDDFS")
solution = iddfs(initial_board)
# for s in solution:
#     s.draw()

print("Depth")
print(len(solution))

Initial Board:
1 | 2 | 3
4 | 5 | 6
7 | 8 | 0
---------
IDDFS


KeyboardInterrupt: 

In [ ]:
def astar_manhattan(initial_board):
    def manhattan_distance(board):
        distance = 0
        for i in range(3):
            for j in range(3):
                value = board.board[i][j]
                if value != 0:
                    # if the value is not 0
                    # calculate the distance between the current position of the value and its goal position
                    distance += abs(i - (value // 3)) + abs(j - (value % 3))
        return distance
    frontier = []
    heapq.heappush(frontier, (manhattan_distance(initial_board), initial_board))
    on_frontier = set([initial_board])
    visited = set()

    while frontier:
        g, current = heapq.heappop(frontier)
        # pop the state with the smallest f value from the frontier
        on_frontier.remove(current)
        visited.add(current)

        if isthegoal(current):
            solution = [current]
            while current.prev:
                solution.append(current.prev)
                current = current.prev
            solution.reverse()
            return solution
        for n in possible_moves(current):
            if n not in visited and n not in on_frontier:
                n.prev = current
                heapq.heappush(frontier, (manhattan_distance(n), n))
                on_frontier.add(n)
    return None


# initial_board = Board([1, 2, 3, 4, 0, 5, 6, 7, 8])
print("Initial Board:")
initial_board.draw()

print("A* Manhattan")
solution = astar_manhattan(initial_board)
# for s in solution:
#     s.draw()

print("Depth")
print(len(solution))

Initial Board:
1 | 2 | 3
4 | 5 | 6
7 | 8 | 0
---------
A* Manhattan
Depth
69


In [ ]:
def astar_euclidean(initial_board):
    def euclidean(board):
        distance =0
        for i in range(3):
            for j in range(3):
                value = board.board[i][j]
                if value != 0:
                    distance += ((value // 3 - i) ** 2 + (value % 3 - j) ** 2) ** 0.5   #####
        return distance
    frontier = []
    heapq.heappush(frontier, (euclidean(initial_board), initial_board))
    on_frontier = set([initial_board])
    visited = set()

    while frontier:
        g, current = heapq.heappop(frontier)
        on_frontier.remove(current)
        visited.add(current)

        if isthegoal(current):
            solution = [current]
            while current.prev:
                solution.append(current.prev)
                current = current.prev
            solution.reverse()
            return solution
        for n in possible_moves(current):
            if n not in visited and n not in on_frontier:
                n.prev = current
                heapq.heappush(frontier, (euclidean(n), n))
                on_frontier.add(n)

    return None
# initial_board = Board([1, 2, 3, 4, 0, 5, 6, 7, 8])
print("Initial Board:")
initial_board.draw()


print("A* Euclidean")
solution = astar_euclidean(initial_board)
# for s in solution:
#     s.draw()

print("Depth")
print(len(solution))

Initial Board:
1 | 2 | 3
4 | 5 | 6
7 | 8 | 0
---------
A* Euclidean
Depth
39
